In [0]:
CREATE OR REPLACE TEMPORARY FUNCTION clean_date(d DATE)
RETURNS DATE
RETURN CASE
  WHEN d IS NULL THEN NULL
  WHEN d < DATE '1901-01-01' THEN NULL
  WHEN d IN (
    DATE '1900-01-01', DATE '1901-01-01',
    DATE '1969-12-31', DATE '1970-01-01',
    DATE '2099-12-31', DATE '2999-12-31', DATE '9999-12-31'
  ) THEN NULL
  WHEN d > current_date() + INTERVAL 3 YEARS THEN NULL
  ELSE d
END;

--CREATE OR REPLACE TABLE teamconnect.pca_prism.14_tbl_trt_events_12m AS
INSERT OVERWRITE TABLE pca_prism.14_tbl_trt_events_12m
WITH trt_scoped AS (
  SELECT
    -- BTN from office/line: trtmex (6), trtmln (1–4)
    CASE 
      WHEN LENGTH(TRIM(CAST(trtmex AS STRING))) = 6
       AND LENGTH(TRIM(CAST(trtmln AS STRING))) BETWEEN 1 AND 4
      THEN CONCAT(
        TRIM(CAST(trtmex AS STRING)),
        LPAD(TRIM(CAST(trtmln AS STRING)), 4, '0')
      )
      ELSE NULL
    END AS btn,
    UPPER(TRIM(env)) AS dpi_environment,

    -- core dates
    clean_date(TRY_TO_DATE(CAST(trtdat AS STRING), 'yyyyMMdd')) AS treatment_date,
    clean_date(TRY_TO_DATE(CAST(trtdue AS STRING), 'yyyyMMdd')) AS treatment_due_date,
    clean_date(TRY_TO_DATE(CAST(trtpdt AS STRING), 'yyyyMMdd')) AS treatment_post_date,

    UPPER(TRIM(trttyp))  AS treatment_type,
    CAST(trtamt AS DECIMAL(9,2)) AS treatment_amount,
    UPPER(TRIM(trtsts))  AS treatment_status
  FROM it_bronze_prod.dpi.bltrt
),
trt_12m AS (
  SELECT *
  FROM trt_scoped
  WHERE btn IS NOT NULL
    AND btn RLIKE '^[0-9]{10}$'
    AND treatment_date IS NOT NULL
    AND treatment_date >= ADD_MONTHS(current_date(), -12)
),
trt_enriched AS (
  SELECT
    btn,
    dpi_environment,
    treatment_date,
    treatment_due_date,
    treatment_post_date,
    treatment_type,
    treatment_amount,
    treatment_status,
    CASE treatment_type
      WHEN 'BPD' THEN 'Bill Past Due (Treatable in WV only)'
      WHEN 'COL' THEN 'Cut Off List'
      WHEN 'CVD' THEN 'COVID-19 Override'
      WHEN 'DFD' THEN 'NY Data Feed'
      WHEN 'DPA' THEN 'Deferred Payment Arrangement'
      WHEN 'DSC' THEN 'Suspend Non-Payment (2-way)'
      WHEN 'DSH' THEN 'Dish Service- Dish Only Suspend'
      WHEN 'EMR' THEN 'Collections Supervisor Use Only for Emergency (i.e. Natural Disasters)'
      WHEN 'MED' THEN 'Medical Override'
      WHEN 'NBS' THEN 'Non-Basic Suspend'
      WHEN 'NRS' THEN 'Non-Regulated Suspend'
      WHEN 'NSF' THEN 'Non-Sufficient Fund Check'
      WHEN 'NVS' THEN 'Video service (FiberOptic or Frontier TV)'
      WHEN 'OWD' THEN 'One-Way Suspend (NY Only)'
      WHEN 'OVR' THEN 'Treatment Override'
      WHEN 'PWO' THEN 'Pre-Write Off'
      WHEN 'PYX' THEN 'Payment Extension'
      WHEN 'REM' THEN 'Reminder Notice'
      WHEN 'RTN' THEN 'Return Notice'
      WHEN 'TBK' THEN 'Toll Suspend'
      WHEN 'TTL' THEN 'Treatment Termination Letter'
      ELSE 'UNKNOWN'
    END AS treatment_type_description
  FROM trt_12m
)
SELECT
  btn,
  dpi_environment,
  COLLECT_LIST(
    named_struct(
      'treatment_date',             treatment_date,
      'treatment_due_date',         treatment_due_date,
      'treatment_post_date',        treatment_post_date,
      'treatment_type',             treatment_type,
      'treatment_type_description', treatment_type_description,
      'treatment_amount',           treatment_amount,
      'treatment_status',           treatment_status
    )
  ) AS treatment_events_last_12m
FROM trt_enriched
GROUP BY btn, dpi_environment;

OPTIMIZE teamconnect.pca_prism.14_tbl_trt_events_12m
ZORDER BY (btn, dpi_environment);